In [2]:
# Nearest Neigbour Analysis

In [6]:
## imports ##
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from sklearn import svm, decomposition, tree
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn import metrics
from sklearn.model_selection import learning_curve, GridSearchCV, cross_val_score, validation_curve
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import numpy as np
import warnings
from sklearn.feature_selection import RFECV, SelectKBest, chi2
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
%matplotlib inline

warnings.simplefilter("ignore")
np.random.seed(100)

In [7]:
## login to synapse
syn = sc.login()

Welcome, aryton tediarjo!



INFO:synapseclient_default:Welcome, aryton tediarjo!



In [12]:
walking_train = pd.read_csv("../Data/walking_data_training.csv", index_col=0)
walking_train_imputed = pd.read_csv("../Data/walking_imputed.csv", index_col=0)
walking_test_imputed = pd.read_csv("../Data/walking_imputed_test_data.csv", index_col=0)
balance_train = pd.read_csv("../Data/balance_data_training.csv", index_col=0).dropna()
# balance_X_train, balance_X_test, balance_y_train, balance_y_test = train_test_split(balance_train.drop(["healthCode", "PD"], axis = 1), balance_train["PD"], test_size=0.20, random_state = 100)
# walking_X_train, walking_X_test, walking_y_train, walking_y_test = train_test_split(walking_train.drop(["healthCode", "PD"], axis = 1), walking_train["PD"], test_size=0.20, random_state = 100)

In [15]:
walking_train.columns

Index(['healthCode', 'max_no_of_steps userAccel_walking_features_AA',
       'max_median_freeze_index userAccel_walking_features_AA',
       'max_count_freeze_index userAccel_walking_features_AA',
       'max_speed_of_gait userAccel_walking_features_AA',
       'max_gait_step_regularity userAccel_walking_features_AA',
       'max_gait_stride_regularity userAccel_walking_features_AA',
       'max_gait_symmetry userAccel_walking_features_AA',
       'max_frequency_of_peaks userAccel_walking_features_AA',
       'max_no_of_steps userAccel_walking_features_x',
       'max_median_freeze_index userAccel_walking_features_x',
       'max_count_freeze_index userAccel_walking_features_x',
       'max_speed_of_gait userAccel_walking_features_x',
       'max_gait_step_regularity userAccel_walking_features_x',
       'max_gait_stride_regularity userAccel_walking_features_x',
       'max_gait_symmetry userAccel_walking_features_x',
       'max_frequency_of_peaks userAccel_walking_features_x',
      

## Healthcode Similarities based on Cosine Similarities


In [72]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors

In [51]:
ix_walking_train = walking_train.set_index("healthCode")
cosine = cosine_similarity(ix_walking_train)
np.fill_diagonal(cosine, 0)
similarity_with_healthcodes = pd.DataFrame(cosine, index= walking_train.set_index("healthCode").index)
similarity_with_healthcodes.columns= walking_train.set_index("healthCode").index

In [55]:
hc_series = similarity_with_healthcodes.loc["0160664a-f4af-4071-a4aa-2967f3ea0503"] 

In [168]:
hc_series[hc_series >= 0.99]

healthCode
2f9c4950-2a3a-49e4-b343-ab6277119ea9    0.990579
821e5fcf-6960-4c50-b370-289e33a55069    0.998189
c562a123-db42-4e70-870c-d9e2df1f7eee    0.993689
Name: 0160664a-f4af-4071-a4aa-2967f3ea0503, dtype: float64

In [169]:
similarity_with_healthcodes.head(3)

healthCode,0160664a-f4af-4071-a4aa-2967f3ea0503,02baa3a2-fb40-4a86-94b4-0411125179f1,036c4104-b14c-4352-82da-7148a2e8f449,05c7ec40-2ea0-4714-910a-d818bd47e503,075e065a-ee03-49ca-9ce1-f68749809e2d,080274a4-cddf-47b7-9b8e-679153859229,09592ab5-605a-4df2-8fec-d21b63f542c7,0a623a36-ea3e-4557-9579-ae960c77f4d7,0e8e18a7-c48d-44a0-9674-3d20fc6218c1,0f5bca0c-0745-4a0d-af3a-e400fad16df2,...,ef22d62b-28f6-49fb-814a-f0d5b7f15171,ef5b9927-f533-4936-9524-e53d2406fac7,f11551a6-a9d5-4226-9772-3ae2fa997b14,f2a19126-3eeb-4588-8290-4c41b8f6582f,f690974e-8179-4564-ab89-1302590861c4,f87dbecf-5af7-4645-b2f0-fd0a2707f0f9,f8cc1925-9a7f-40d2-bd56-2511350c3315,fa735991-ec53-4763-9fa1-2fb499e3efcb,fd8938df-cccd-4cbe-b7cf-fa7102147481,ffab2631-dfe1-4f5e-bd9e-f763e8aede19
healthCode,,,,,,,,,,,,,,,,,,,,,
0160664a-f4af-4071-a4aa-2967f3ea0503,0.000000,0.079193,0.262604,0.455220,0.271319,0.249629,0.090041,0.280175,0.836364,0.259493,...,0.770036,0.575837,0.245305,0.207750,0.496018,0.403545,0.854992,0.365201,0.881506,0.228091
02baa3a2-fb40-4a86-94b4-0411125179f1,0.079193,0.000000,0.073749,0.042854,0.184440,0.982568,0.992032,0.568055,0.099876,0.422639,...,0.329466,0.550183,0.640001,0.303653,0.466174,0.362790,0.531557,0.248330,0.537335,0.491011
036c4104-b14c-4352-82da-7148a2e8f449,0.262604,0.073749,0.000000,0.973310,0.983316,0.092327,0.189075,0.276381,0.087822,0.157890,...,0.691647,0.756733,0.813320,0.307301,0.832063,0.898168,0.350949,0.252298,0.309011,0.330152


In [241]:
def getKNN(data, no_neighbors):
    data = data.set_index("healthCode")
    min_max_scaler = MinMaxScaler()
    walking_features = min_max_scaler.fit_transform(data)
    nbrs = NearestNeighbors(n_neighbors = 5, 
                            algorithm = 'auto', metric = "cosine").fit(walking_features)
    distance, indices = nbrs.kneighbors(walking_features)
    return distance, indices

In [242]:
distance, indices = getKNN(walking_train, 3)

In [245]:
dict_ = {}
for i in indices:
    dict_[walking_train.ix[i[0]]["healthCode"]] = [_ for _ in walking_train.ix[i[1:]]["healthCode"]]
data = (pd.DataFrame(dict_,).T)

In [246]:
data

,0,1,2,3
0160664a-f4af-4071-a4aa-2967f3ea0503,74de3c85-715c-4962-9cd7-28bdf4e96a73,90550db0-af30-495f-8d7c-5749577cdae2,b3404b35-b8f8-4f14-afc6-92f0d1bdf920,b4bb8c67-33ac-4333-b1aa-6b79babc40fc
02baa3a2-fb40-4a86-94b4-0411125179f1,1c3d5685-0354-42fc-b882-cf2d37be1e49,e9d63346-1c6a-40b9-9fd7-5ddc308b45e6,c8fe27c1-2e47-42a5-a087-e02d40d0408b,20b09fcb-eb66-4282-b53a-51aa24f1f9d2
036c4104-b14c-4352-82da-7148a2e8f449,4b29f163-61b9-4fce-87be-79688e54b37d,e2e75720-d0a2-4b73-b93f-0d6f97a7acf3,9a41dd95-337d-4f23-8b3e-f0f0dd40fc4d,cbcbdf73-e3ee-48b1-a2e8-80ee5d37b722
05c7ec40-2ea0-4714-910a-d818bd47e503,c128d96b-4c6c-434e-9fe7-111096000775,b72761a5-1b95-4a71-bf37-4d3364e07bd7,e0c07194-9e79-4674-868d-b05d480b90a3,b562b352-4ce7-4754-a2c3-1d799bc5fb33
075e065a-ee03-49ca-9ce1-f68749809e2d,0e8e18a7-c48d-44a0-9674-3d20fc6218c1,3f68da18-e3d6-426b-801f-9d8ccd46e6cb,65981245-0d78-486c-8466-d705c022920f,6c130d05-41af-49e9-9212-aaae738d3ec1
080274a4-cddf-47b7-9b8e-679153859229,ef22d62b-28f6-49fb-814a-f0d5b7f15171,cd970c0a-5414-4a79-8778-9ecd7c377584,3d4d0805-96b7-4b27-86fa-55638def105a,36ca2d1a-cf6f-4730-bdd4-ed8000409707
09592ab5-605a-4df2-8fec-d21b63f542c7,821e5fcf-6960-4c50-b370-289e33a55069,cd970c0a-5414-4a79-8778-9ecd7c377584,ef22d62b-28f6-49fb-814a-f0d5b7f15171,c8fe27c1-2e47-42a5-a087-e02d40d0408b
0a623a36-ea3e-4557-9579-ae960c77f4d7,873c1263-d757-4dbf-9b48-facf75ab003e,8b7badff-6955-47ce-9867-c170f7d79dbd,2911f2f3-feb8-4de5-9ef9-3685e1fdbfa3,fd8938df-cccd-4cbe-b7cf-fa7102147481
0e8e18a7-c48d-44a0-9674-3d20fc6218c1,075e065a-ee03-49ca-9ce1-f68749809e2d,81434ce9-c1db-4717-81e4-f49b0942e321,8baf61c8-1ad8-44e0-86c0-f29c6d564cb5,3f68da18-e3d6-426b-801f-9d8ccd46e6cb
0f5bca0c-0745-4a0d-af3a-e400fad16df2,15b05838-6779-4cb9-bc53-86571636c633,f11551a6-a9d5-4226-9772-3ae2fa997b14,cbcbdf73-e3ee-48b1-a2e8-80ee5d37b722,2f59d408-6513-4d04-b8d3-3a3d68d31880
